In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
train = pd.read_csv('../MNIST_data/train.csv')

In [3]:
train_x = (train.iloc[:, 1:] - 128) / 256

In [4]:
train_y_1d = train.iloc[:, 0]

In [5]:
train_y = np.zeros((train_x.shape[0], 10))

In [6]:
train_y[list(range(train_x.shape[0])), train_y_1d] = 1

In [7]:
train_y[:5]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [8]:
train_y_1d.head()

0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [9]:
row_ids = list(range(train_x.shape[0] - 2000))

In [10]:
batch_size = 1000

## Train Saved Model

In [11]:
saver = tf.train.import_meta_graph('saved_models/simple_nn_model.meta')

In [12]:
sess = tf.Session()

In [13]:
saver.restore(sess=sess, save_path=tf.train.latest_checkpoint('saved_models/'))

INFO:tensorflow:Restoring parameters from saved_models/simple_nn_model


In [14]:
graph = tf.get_default_graph()

In [15]:
x = graph.get_tensor_by_name('input:0')
y = graph.get_tensor_by_name('label:0')
train_step = graph.get_operation_by_name('train_step')
accuracy = graph.get_tensor_by_name('accuracy:0')

In [16]:
for _ in range(5):
    if _ % 10000 == 0: print(_)
    batch_row_ids = np.random.choice(row_ids, size=batch_size, replace=False)
    train_step.run(session=sess, feed_dict={x: train_x.iloc[batch_row_ids], y: train_y[batch_row_ids, :]})

0


In [18]:
print(accuracy.eval(session=sess, feed_dict={x: train_x.iloc[-2000:], y: train_y[-2000:]}))

0.1225


## Save Retrained Model

In [20]:
saver = tf.train.Saver()
saver.save(sess, 'saved_models/simple_nn_model')
sess.close()